In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import re
from time import sleep
import random
#return a randomly generated number
#from a normal distribution
#but within a range
#source: https://stackoverflow.com/a/26920781
def trunc_gauss(mu, sigma, bottom, top):
    a = random.gauss(mu,sigma)
    while (bottom <= a <= top) == False:
        a = random.gauss(mu,sigma)
    return a

#function to sleep a random amount of time, mean 2.5, sd 0.5, but no less than 2.0001 or more than 2.9999 seconds
#this is necessary because if the wait time is too high, the connection between selenium and the browser might break
def short_sleep():
    sleep(trunc_gauss(2.5, 0.5, 2.0001, 2.9999))

#load the data detailing the senators' facebook accounts
df_senators = pd.read_csv("../data/senateSocialMedia.csv", encoding='latin1')
df_senators['verifiedFacebook'] = "NA"
#df_house = pd.read_csv("../data/houseSocialMedia.csv", encoding='latin1')
#df_house['verifiedFacebook'] = "NA"

#loop over the different senators
for j in range(0,len(df_senators)):
    
    driver = webdriver.Firefox()

    #go to the senator's faceboom page
    driver.get(df_senators['facebook'][j])
    #record the actual URL, which might differ from the one given (in case of a redirect)
    df_senators['verifiedFacebook'][j] = driver.current_url
    
    try:
        #find the button that says 'Videos'
        video_link = driver.find_element_by_link_text('Videos')
        #click on it
        video_link.click()
        short_sleep()

        #currently not used, but possibly useful: manually moving the mouse cursor from one element to somewhere else and then clicking there
        #define an action that does the following:
        #move the mouse cursor to the Create New Account button, go up 100 units and click
        #actions = ActionChains(driver)
        #actions.move_to_element_with_offset(driver.find_element_by_link_text('Create New Account'), 0,0)
        #actions.move_by_offset(0,-100).click()
        #use that action:
        #action.perform()

        #scroll down and click on the Show More button five times
        for i in range(0,5):
        #alternatively, do it until there is no more Show More button:
        #while True:
            try:
                #scroll all the way down and wait
                driver.find_element_by_tag_name('a').send_keys(Keys.CONTROL + Keys.END)
                short_sleep()
                #click on show more
                driver.find_element_by_link_text('Show More').click()
                short_sleep()
            #if Facebook puts up one of the "log in to continue" elements, the show more button will be obscured
            except ElementClickInterceptedException as exception:
                driver.find_element_by_link_text('Not Now').click()
                short_sleep()
                driver.find_element_by_tag_name('a').send_keys(Keys.CONTROL + Keys.END)
                short_sleep()
            #exception for when at the bottom of the page and there is now Show More button
            except NoSuchElementException as exception:
                break


        #wait a bit for javascript
        short_sleep()
        #get the inner HTML (i.e. with javascript)
        innerHTML = driver.execute_script("return document.body.innerHTML")
        #find all videos with regex
        urls = re.findall(df_senators['verifiedFacebook'][j].replace("https://www.facebook.com/","") + "/videos/[0-9]+/", innerHTML)
        #unique urls
        urls = list(set(urls))

        #write video urls to file
        filename = "../data/facebookVideoURLsSenate/" + df_senators['state'][j] + "_" + df_senators['name'][j].replace(" ", "_") + ".txt"
        with open(filename, 'w') as f:
            for item in urls:
                f.write("%s\n" % item)

        #write html to file just in case I want to parse them in more detail later
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(innerHTML)
        Html_file = open("../data/facebookVideoURLsSenate/" + df_senators['state'][j] + "_" + df_senators['name'][j].replace(" ", "_") + ".html","w")
        Html_file.write(str(soup.encode('latin1')))
        Html_file.close()
        
        driver.close()
    
    #a couple of Senators' FB pages just don't work for whatever reason, so skip past those
    except NoSuchElementException as exception:
        continue

C:\Users\Markus\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Markus\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import re
from time import sleep
import random
#return a randomly generated number
#from a normal distribution
#but within a range
#source: https://stackoverflow.com/a/26920781
def trunc_gauss(mu, sigma, bottom, top):
    a = random.gauss(mu,sigma)
    while (bottom <= a <= top) == False:
        a = random.gauss(mu,sigma)
    return a

#function to sleep a random amount of time, mean 2.5, sd 0.5, but no less than 2.0001 or more than 2.9999 seconds
#this is necessary because if the wait time is too high, the connection between selenium and the browser might break
def short_sleep():
    sleep(trunc_gauss(2.5, 0.5, 2.0001, 2.9999))

site = "https://www.cityofdenhamsprings.com/"

driver = webdriver.Firefox()
driver.get(site)
elems = driver.find_elements_by_xpath("//a[@href]")
hrefs = []
for elem in elems:
    hrefs.append(elem.get_attribute("href"))

In [2]:
import tldextract
#check the domain name of the website's homepage
site_tld = tldextract.extract(site)

In [3]:
#check each of the urls found on the site for whether it actually belongs to the same domain
#gather the ones that do in the site_links list
site_links = []
for href in hrefs:
    href_tld = tldextract.extract(href)
    if href_tld.domain == site_tld.domain:
        site_links.append(href)

In [4]:
#remove duplicates; requires python 3.7
site_links = list(dict.fromkeys(site_links))

In [7]:
#kick out email addresses
site_links = [i for i in site_links if not '@' in i]
#which URLs have we already visited?
urls_visited = dict.fromkeys(site_links, False)

In [9]:
new_links = []

#keep going over the dict until all links have been explored
while not all(value == True for value in urls_visited.values()):

    for i in urls_visited:

        if urls_visited[i]==False:

            #go to the subdomain url
            #and extract all the urls again
            driver.get(i)
            elems = driver.find_elements_by_xpath("//a[@href]")
            hrefs = []
            for elem in elems:
                hrefs.append(elem.get_attribute("href"))
            #sub_site_links = []
            for href in hrefs:
                href_tld = tldextract.extract(href)
                if href_tld.domain == site_tld.domain:
                    new_links.append(href)

            #note that we have visited the url        
            urls_visited[i] = True

            #add all the new links
            #for link in sub_site_links:
            #    urls_visited.setdefault(link, 'False')

    #remove duplicates
    new_links = list(dict.fromkeys(new_links))
    #remove email addresses
    new_links = [i for i in new_links if not '@' in i]

    for i in new_links:
        #potentially remove a # at the end of the url (preventing duplicates)
        #and then add to the dictionary
        urls_visited.setdefault(re.sub('#$', '', i), 'False')

StaleElementReferenceException: Message: The element reference of <a href="https://www.facebook.com/cityofds/"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
